<a href="https://colab.research.google.com/github/moodlep/rl-playground/blob/main/ppo/colab_notebooks/PPO_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]

In [ ]:
!pip install stable-baselines3

In [ ]:
import os
import Box2D
import pyglet
import imageio
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

# Test Env

In [ ]:
env_id = "LunarLanderContinuous-v2"
env = gym.make(env_id)


In [ ]:
env.reset()

array([-0.00715675,  1.4082968 , -0.7249219 , -0.11660757,  0.00829972,
        0.16420564,  0.        ,  0.        ], dtype=float32)

In [ ]:
env.action_space.shape, env.observation_space.shape

((2,), (8,))

In [ ]:
for episode in range(1): 
    observation = env.reset()
    for step in range(1):
        action = env.action_space.sample()  # or given a custom model, action = policy(observation)
        observation, reward, done, info = env.step(action)
        print(observation, reward, done, info, action)

[-0.00568371  1.390598   -0.2911852  -0.44634733  0.0060324   0.05562949
  0.          0.        ] 1.0464224154058968 False {} [0.06701595 0.15616083]


In [ ]:
from stable_baselines3.common.vec_env import SubprocVecEnv

# First skeleton

In [ ]:
class ActorCritic(nn.Module):

  def __init__(self, state_dim, action_dim, std_init):
    super(ActorCritic,self).__init__()

    # TBD switch to variable std

    self.action_dim = action_dim
    self.state_dim = state_dim
    self.critic = nn.Sequential(
        nn.Linear(self.state_dim, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, 1)
    )

    self.actor = nn.Sequential(
        nn.Linear(self.state_dim, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, self.action_dim),
        nn.Tanh()
    )

    # covariance for Multivariate Normal policy
    self.action_vars = torch.full((self.action_dim,), std_init * std_init)
    self.cov_mat = torch.diag(self.action_vars).unsqueeze(dim=0)  # do we need the unsqueeze? 

  def get_action(self, state):
    means = self.actor(state)
    policy = torch.distributions.MultivariateNormal(means, self.cov_mat)

    return policy.sample()

    
  def get_value(self,state):
    return self.critic(state)

  



In [ ]:
ac = ActorCritic(env.observation_space.shape[0], env.action_space.shape[0], 0.05)

In [ ]:
states = torch.rand

In [ ]:
pi = ac.get_action(torch.tensor([observation, observation]))

# Buffer

In [ ]:
class Buffer:
  def __init__(self,state_dim, action_dim ):
    #self.num_actors = num_actors
    #self.max_timesteps = max_timestep
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.buffer = torch.zeros_like([])
  def add_transion()

# Envs

In [ ]:
def make_env(env_id: str, rank: int, seed: int = 0):
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    set_random_seed(seed)
    return _init



num_cpu = 4
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

## covariance stuff

Src: https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/PPO_colab.ipynb

In [ ]:
torch.full((2,), 3 * 3)

tensor([9, 9])

In [ ]:
torch.diag(torch.full((2,), 3 * 3)).unsqueeze(dim=0)

tensor([[[9, 0],
         [0, 9]]])